In [1]:
# Importing the libraries
from pathlib import Path

import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

2022-09-01 06:57:43.538785: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-01 06:57:43.621138: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-01 06:57:43.621149: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [37]:
base_csv = Path('data/clean_frame.csv')
test_csv = Path('data/test_frame.csv')

In [38]:
base_df = pd.read_csv(base_csv, index_col='ind')
base_df.columns

Index(['ID', 'Код_группы', 'Год_Поступления', 'Год_Окончания_УЗ', 'Пособие',
       'Общежитие', 'Наличие_Матери', 'Наличие_Отца', 'Опекунство', 'Село',
       'Иностранец', 'КодФакультета', 'СрБаллАттестата', 'Статус', 'male',
       'female', 'sex_summ', 'birth_year_int', 'basis', 'language', 'country',
       'region', 'city', 'parents_country'],
      dtype='object')

In [39]:
test_df = pd.read_csv(test_csv, index_col='ind')
test_df.columns

Index(['ID', 'Код_группы', 'Год_Поступления', 'Год_Окончания_УЗ', 'Пособие',
       'Общежитие', 'Наличие_Матери', 'Наличие_Отца', 'Опекунство', 'Село',
       'Иностранец', 'КодФакультета', 'СрБаллАттестата', 'male', 'female',
       'sex_summ', 'birth_year_int', 'Статус', 'basis', 'language', 'country',
       'parents_country'],
      dtype='object')

In [4]:
base_df = base_df.fillna(0)


In [42]:
#test_df.isna().any()
test_df = test_df.fillna(0)

In [43]:
columns = ['Наличие_Матери', 'Село', 'Иностранец', 'СрБаллАттестата', 'Статус', 'male', 'basis','language','country', 'КодФакультета', 'Общежитие', 'parents_country']
#columns = base_df.columns.to_list()
priznaki = len(columns)
priznaki

12

In [44]:
frame_to_train = base_df.loc[:, columns]
frame_to_test = test_df.loc[:, columns]

In [45]:
result_key = {
    -1: 0,
    3: 1,
    4: 2,
}

In [8]:
for key, value in result_key.items():
    frame_to_train.loc[frame_to_train['Статус'] == key, 'Статус'] = value

frame_to_train.groupby('Статус').count()

,Наличие_Матери,Село,Иностранец,СрБаллАттестата,male,basis,language,country,КодФакультета,Общежитие,parents_country
Статус,,,,,,,,,,,
0,614,614,614,614,614,614,614,614,614,614,614
1,4721,4721,4721,4721,4721,4721,4721,4721,4721,4721,4721
2,8249,8249,8249,8249,8249,8249,8249,8249,8249,8249,8249


In [9]:
train, test = train_test_split(frame_to_train, test_size=0.20)

In [10]:
train_input = train.drop('Статус', axis=1)
train_output = train['Статус']

In [11]:
test_input = test.drop('Статус', axis=1)
test_output = test['Статус']

In [12]:
train_output.info

<bound method Series.info of ind
7193     2
13465    2
2694     2
7620     1
2382     2
        ..
5036     1
5271     2
2342     1
13560    2
12691    1
Name: Статус, Length: 10867, dtype: int64>

In [13]:
input_df_size = train_input.shape[0]

In [14]:
# Constructing the input
x = train_input.to_numpy()
y = train_output.values
arr = np.transpose(x)

In [15]:
output = np.ones((1,input_df_size))
output[0] = y
arr_y = output.reshape(input_df_size,1)
arr_y

array([[2.],
       [2.],
       [2.],
       ...,
       [1.],
       [2.],
       [1.]])

In [16]:
# Definig the model
model_better = keras.Sequential([
    keras.layers.Dense(150, input_shape=(priznaki-1,), activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(250, activation='relu'),    
    keras.layers.Dense(3, activation='softmax'),
])

2022-09-01 06:57:46.085594: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-01 06:57:46.085610: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-01 06:57:46.085623: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (astra): /proc/driver/nvidia/version does not exist
2022-09-01 06:57:46.086146: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
# Compiling the model
model_better.compile(optimizer='Nadam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

In [18]:
# fitting the model
model_better.fit(x, y, epochs=100, batch_size=70)

Epoch 1/100
156/156 [==============================] - 1s 789us/step - loss: 0.8061 - accuracy: 0.6063
Epoch 2/100
156/156 [==============================] - 0s 736us/step - loss: 0.7828 - accuracy: 0.6219
Epoch 3/100
156/156 [==============================] - 0s 754us/step - loss: 0.7703 - accuracy: 0.6317
Epoch 4/100
156/156 [==============================] - 0s 826us/step - loss: 0.7631 - accuracy: 0.6401
Epoch 5/100
156/156 [==============================] - 0s 712us/step - loss: 0.7529 - accuracy: 0.6422
Epoch 6/100
156/156 [==============================] - 0s 716us/step - loss: 0.7495 - accuracy: 0.6478
Epoch 7/100
156/156 [==============================] - 0s 701us/step - loss: 0.7428 - accuracy: 0.6509
Epoch 8/100
156/156 [==============================] - 0s 738us/step - loss: 0.7408 - accuracy: 0.6528
Epoch 9/100
156/156 [==============================] - 0s 773us/step - loss: 0.7329 - accuracy: 0.6595
Epoch 10/100
156/156 [==============================] - 0s 757us/step - l

In [19]:
# Evaluating the model
x_t = test_input.to_numpy()
y_t = test_output.values

model_better.evaluate(x_t, y_t, batch_size=8)

340/340 [==============================] - 0s 521us/step - loss: 0.7305 - accuracy: 0.6831


[0.7305381298065186, 0.6831063628196716]

In [20]:
bet = model_better.evaluate(x_t, y_t, batch_size=8)
bet

340/340 [==============================] - 0s 468us/step - loss: 0.7305 - accuracy: 0.6831


[0.7305381298065186, 0.6831063628196716]

In [21]:
tusa = model_better.predict(x_t)
tusa

85/85 [==============================] - 0s 458us/step


array([[0.0535363 , 0.41751042, 0.5289532 ],
       [0.00586   , 0.8762062 , 0.11793387],
       [0.09478706, 0.455407  , 0.44980592],
       ...,
       [0.0974069 , 0.4196796 , 0.48291343],
       [0.08341489, 0.56166714, 0.35491797],
       [0.00165133, 0.71056443, 0.28778425]], dtype=float32)

In [22]:
def get_result(answer_arr: np.array) -> int:
    answer_code = np.argmax(answer_arr)

    # list out keys and values separately
    key_list = list(result_key.keys())
    val_list = list(result_key.values())
    
    # print key with val 100
    position = val_list.index(answer_code)
    return  key_list[position], answer_code

In [23]:
answers = []
for answer_arr in tusa:
    answers.append(get_result(answer_arr)[1])

In [24]:
test_output

ind
4754     2
4274     2
9568     1
222      2
13209    1
        ..
5430     0
1576     1
1733     1
9931     2
5246     2
Name: Статус, Length: 2717, dtype: int64

In [25]:
compare_df = pd.DataFrame(columns=['real', 'calculation'])
compare_df['real'] = y_t
compare_df['calculation'] = answers
compare_df

,real,calculation
0,2,2
1,2,1
2,1,1
3,2,2
4,1,2
...,...,...
2712,0,2
2713,1,2
2714,1,2
2715,2,1


In [26]:
compare_df['new'] = np.where((compare_df['real'] == compare_df['calculation']), compare_df['real'], 100)
compare_df.groupby('new').count()

,real,calculation
new,,
1,496,496
2,1360,1360
100,861,861


In [27]:
r = 1 + 386 + 1032 + 320
620 / r

0.35652673950546293

# Чтение и предсказие test dataset

In [48]:
train_input = train.drop('Статус', axis=1)
input_df_size = train_input.shape[0]

In [49]:
# Constructing the input
x_test = train_input.to_numpy()

In [58]:
predict_arr = model_better.predict(x)
predict_arr

340/340 [==============================] - 0s 432us/step


array([[1.15134381e-01, 4.93934751e-01, 3.90930802e-01],
       [2.08177250e-02, 2.95132577e-01, 6.84049606e-01],
       [1.19731305e-02, 2.76902556e-01, 7.11124301e-01],
       ...,
       [1.27114967e-01, 7.78385222e-01, 9.44997370e-02],
       [4.93095395e-05, 1.02611646e-01, 8.97339046e-01],
       [1.20784370e-02, 6.67284369e-01, 3.20637226e-01]], dtype=float32)

In [59]:
answers = []
for answer_arr in predict_arr:
    answers.append(get_result(answer_arr)[1])
answers

[1,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,


In [ ]:
compare_df = pd.DataFrame(columns=['real', 'calculation'])
compare_df['real'] = y
compare_df['calculation'] = answers
compare_df